# Cleaning the OULAD data

In [1]:
import pandas as pd
import numpy as np
 
from sqlalchemy import create_engine,text
user = 'root'
passwords = '7003890541'
host = 'localhost'
port = 3306
database = 'open_university'

engine = create_engine(f"mysql+pymysql://{user}:{passwords}@{host}:{port}/{database}")
def sql(query , engine = engine):
    return pd.read_sql(query,engine)


def select(table_name):
    return sql(f'select * from {table_name} limit 5')


def run(query):
    with engine.connect() as con:
        con.execute(text(query))
        con.commit()

In [2]:
query = 'show tables in open_university'
tables = sql(query)
table_names = [col[0] for col in tables.values.tolist()]

In [9]:
# for names in table_names:
#     run(f'''
#         create table backup_{names} as
#         select * from {names};
#     ''')

In [10]:
select('assessments')

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


In [13]:
query = f'''
    alter table assessments
    rename column code_module to course_name,
    rename column code_presentation to course_part,
    rename column id_assessment to assessment_id,
    rename column date to days
'''

run(query)

In [14]:
select('assessments')

,course_name,course_part,assessment_id,assessment_type,days,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


In [15]:
select('courses')

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


In [16]:
query = '''
    alter table courses
    rename column code_module to course_name,
    rename column code_presentation to course_part,
    rename column module_presentation_length to part_length
'''
run(query)

In [17]:
select('courses')

,course_name,course_part,part_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


In [18]:
select('student_assessment')

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0


In [19]:
query = '''
    alter table student_assessment
    rename column id_assessment to assessment_id,
    rename column id_student to student_id,
    rename column date_submitted to submitted_after
'''
run(query)

In [20]:
select('student_assessment')

,assessment_id,student_id,submitted_after,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0


In [30]:
select('student_info')

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass


In [22]:
query = '''
    create table students as
    select distinct 
    id_student as student_id,
    gender,region,
    highest_education as education,
    imd_band,age_band,disability
    from student_info;
'''
run(query)

In [23]:
select('students')

,student_id,gender,region,education,imd_band,age_band,disability
0,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,N
1,28400,F,Scotland,HE Qualification,20-30%,35-55,N
2,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,Y
3,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,N
4,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,N


In [31]:
sql('select count(distinct student_id) from students')

,count(distinct student_id)
0,28785


In [32]:
query = '''
    alter table student_info
    drop column gender,
    drop column region,
    drop column highest_education,
    drop column imd_band,
    drop column age_band,
    drop column disability
'''
run(query)

In [39]:
select('student_info')

,course_name,part_name,student_id,previous_attempts,studied_credits,final_result
0,AAA,2013J,11391,0,240,Pass
1,AAA,2013J,28400,0,60,Pass
2,AAA,2013J,30268,0,60,Withdrawn
3,AAA,2013J,31604,0,60,Pass
4,AAA,2013J,32885,0,60,Pass


In [ ]:
query = '''
    alter table student_info
    rename column code_module to course_name,
    rename column code_presentation to part_name,
    rename column id_student to student_id,
    rename column num_of_prev_attempts to previous_attempts
'''
run(query)

In [34]:
select('student_registration')

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN


In [41]:
query = '''
    alter table student_registration
    rename column code_module to course_name,
    rename column code_presentation to part_name,
    rename column id_student to student_id,
    rename column date_registration to registration_day,
    rename column date_unregistration to unregister_day
'''
run(query)

In [43]:
select('student_registration')

,course_name,part_name,student_id,registration_day,unregister_day
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN


In [44]:
select('student_vle')

,code_module,code_presentation,id_student,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1


In [45]:
query = '''
    alter table student_vle
    rename column code_module to course_name,
    rename column code_presentation to part_name,
    rename column id_student to student_id
'''

run(query)

In [46]:
select('student_vle')

,course_name,part_name,student_id,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1


In [47]:
select('vle')

,id_site,code_module,code_presentation,activity_type,week_from,week_to
0,546943,AAA,2013J,resource,None,None
1,546712,AAA,2013J,oucontent,None,None
2,546998,AAA,2013J,resource,None,None
3,546888,AAA,2013J,url,None,None
4,547035,AAA,2013J,resource,None,None


In [48]:
query = '''
    alter table vle
    rename column id_site to site_id,
    rename column code_module to course_name,
    rename column code_presentation to part_name
'''
run(query)

# _____________________________________________________________________

In [50]:
select('assessments')

,course_name,course_part,assessment_id,assessment_type,days,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


In [51]:
select('courses')

,course_name,course_part,part_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


`ASSESSMENT IS CONNECTED TO COURSES USING **COURSE_NAME** ANDE **COURSE_PART** COLUMNS`

In [54]:
select('students')

,student_id,gender,region,education,imd_band,age_band,disability
0,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,N
1,28400,F,Scotland,HE Qualification,20-30%,35-55,N
2,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,Y
3,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,N
4,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,N


In [52]:
select('student_assessment')

,assessment_id,student_id,submitted_after,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0


`STUDENTS AND STUDENTSASSESSMENT ARE CONNECTED USING **STUDENT ID**`

In [55]:
select('student_info')

,course_name,part_name,student_id,previous_attempts,studied_credits,final_result
0,AAA,2013J,11391,0,240,Pass
1,AAA,2013J,28400,0,60,Pass
2,AAA,2013J,30268,0,60,Withdrawn
3,AAA,2013J,31604,0,60,Pass
4,AAA,2013J,32885,0,60,Pass


`RECORDS OF EACH STUDENT, THEIR FINAL RESULT, COURSE NAME AND PART NAME...CAN BE TREATED AS MAIN TABLE`

In [56]:
select('student_registration')

,course_name,part_name,student_id,registration_day,unregister_day
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN


`REGISTRATION DETAILS OF THE STUDENTS, AND ALSO MOST IMPORTANTLY THE UNREGISTRATION DAY, IF POSSIBLE`

In [57]:
select('vle')

,site_id,course_name,part_name,activity_type,week_from,week_to
0,546943,AAA,2013J,resource,None,None
1,546712,AAA,2013J,oucontent,None,None
2,546998,AAA,2013J,resource,None,None
3,546888,AAA,2013J,url,None,None
4,547035,AAA,2013J,resource,None,None


`VLE DETAILS, WHICH COURSE AND PART IT BELONGS TO AND THE START AND END DATE`

In [58]:
select('student_vle')

,course_name,part_name,student_id,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1


`INFORMATION ABOUT WHICH STUDENT OF WHICH COURSE AND PART TRIED TO ACCESS MATERIAL AND HOW MANY TIMES`

**Function for fetching all columns**

In [60]:
def fetch_all_cols(table):
    query = f'''
        select column_name from information_schema.columns
        where table_name = '{table}' and
        table_schema = database()
    '''
    df = sql(query)
    return df['COLUMN_NAME'].values.tolist()

**Find null values**

In [61]:
def find_nulls(table):
    dict ={}
    for col in fetch_all_cols(table):
        query = f'''
            select sum({col} is null) as {col}_nulls from {table}
        '''
        df =sql(query)
        dict[col] = df.iloc[0].values.astype('int')
    display(pd.DataFrame.
            from_dict(dict,orient='index',columns=['null_values']).
            reset_index().
            rename(columns={'index':'column_names'}))

In [88]:
table_names = ['assessments', 'courses', 'student_assessment', 'student_info', 'student_registration', 'student_vle', 'vle','students']

In [71]:
select('student_info')

,course_name,part_name,student_id,previous_attempts,studied_credits,final_result
0,AAA,2013J,11391,0,240,Pass
1,AAA,2013J,28400,0,60,Pass
2,AAA,2013J,30268,0,60,Withdrawn
3,AAA,2013J,31604,0,60,Pass
4,AAA,2013J,32885,0,60,Pass


In [72]:
find_nulls('student_info')

,column_names,null_values
0,course_name,0
1,part_name,0
2,student_id,0
3,previous_attempts,0
4,studied_credits,0
5,final_result,0


In [73]:
select('assessments')

,course_name,course_part,assessment_id,assessment_type,days,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


In [75]:
find_nulls('assessments')

,column_names,null_values
0,course_name,0
1,course_part,0
2,assessment_id,0
3,assessment_type,0
4,days,11
5,weight,0


In [76]:
query = '''
    update assessments
    set days = 0
    where days is null;
'''
run(query)

In [78]:
select('courses')

,course_name,course_part,part_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


In [79]:
find_nulls('courses')

,column_names,null_values
0,course_name,0
1,course_part,0
2,part_length,0


In [80]:
select('student_assessment')

,assessment_id,student_id,submitted_after,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0


In [82]:
find_nulls('student_assessment')

,column_names,null_values
0,assessment_id,0
1,student_id,0
2,submitted_after,0
3,is_banked,0
4,score,173


In [84]:
query = '''
    update student_assessment
    set score = 0
    where score is null;
'''
run(query)

In [85]:
find_nulls('student_assessment')

,column_names,null_values
0,assessment_id,0
1,student_id,0
2,submitted_after,0
3,is_banked,0
4,score,0


In [86]:
select('student_info')

,course_name,part_name,student_id,previous_attempts,studied_credits,final_result
0,AAA,2013J,11391,0,240,Pass
1,AAA,2013J,28400,0,60,Pass
2,AAA,2013J,30268,0,60,Withdrawn
3,AAA,2013J,31604,0,60,Pass
4,AAA,2013J,32885,0,60,Pass


In [87]:
find_nulls('student_info')

,column_names,null_values
0,course_name,0
1,part_name,0
2,student_id,0
3,previous_attempts,0
4,studied_credits,0
5,final_result,0


In [89]:
select('student_registration')

,course_name,part_name,student_id,registration_day,unregister_day
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN


In [90]:
find_nulls('student_registration')

,column_names,null_values
0,course_name,0
1,part_name,0
2,student_id,0
3,registration_day,45
4,unregister_day,22521


In [91]:
query = '''
    update student_registration
    set registration_day = 0
    where registration_day is null;
'''
run(query)

In [92]:
query = '''
    update student_registration
    set unregister_day = 0
    where unregister_day is null;
'''
run(query)

In [93]:
find_nulls('student_registration')

,column_names,null_values
0,course_name,0
1,part_name,0
2,student_id,0
3,registration_day,0
4,unregister_day,0


In [94]:
select('vle')

,site_id,course_name,part_name,activity_type,week_from,week_to
0,546943,AAA,2013J,resource,None,None
1,546712,AAA,2013J,oucontent,None,None
2,546998,AAA,2013J,resource,None,None
3,546888,AAA,2013J,url,None,None
4,547035,AAA,2013J,resource,None,None


In [96]:
sql('select count(*) from vle')

,count(*)
0,6364


In [99]:
sql('select * from vle where week_from is not null limit 5')

,site_id,course_name,part_name,activity_type,week_from,week_to
0,546732,AAA,2013J,oucontent,2.0,2.0
1,546719,AAA,2013J,oucontent,1.0,1.0
2,546681,AAA,2013J,oucontent,1.0,1.0
3,877040,AAA,2014J,oucontent,2.0,2.0
4,877045,AAA,2014J,oucontent,1.0,1.0


In [95]:
find_nulls('vle')

,column_names,null_values
0,site_id,0
1,course_name,0
2,part_name,0
3,activity_type,0
4,week_from,5243
5,week_to,5243


In [100]:
query = '''
    update vle
    set week_from = 0
    where week_from is null;
'''
run(query)

In [101]:
query = '''
    update vle
    set week_to = 0
    where week_to is null;
'''
run(query)

In [102]:
find_nulls('vle')

,column_names,null_values
0,site_id,0
1,course_name,0
2,part_name,0
3,activity_type,0
4,week_from,0
5,week_to,0


In [103]:
select('student_vle')

,course_name,part_name,student_id,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1


In [104]:
find_nulls('student_vle')

,column_names,null_values
0,course_name,0
1,part_name,0
2,student_id,0
3,id_site,0
4,date,0
5,sum_click,0


# _____________________________________________________________________

`STUDENT INFO IS CONSIDERED AS THE MAIN TABLE`

In [105]:
select('student_info')

,course_name,part_name,student_id,previous_attempts,studied_credits,final_result
0,AAA,2013J,11391,0,240,Pass
1,AAA,2013J,28400,0,60,Pass
2,AAA,2013J,30268,0,60,Withdrawn
3,AAA,2013J,31604,0,60,Pass
4,AAA,2013J,32885,0,60,Pass


In [106]:
student_info = sql('select * from student_info')

In [107]:
student_info.head(5)

,course_name,part_name,student_id,previous_attempts,studied_credits,final_result
0,AAA,2013J,11391,0,240,Pass
1,AAA,2013J,28400,0,60,Pass
2,AAA,2013J,30268,0,60,Withdrawn
3,AAA,2013J,31604,0,60,Pass
4,AAA,2013J,32885,0,60,Pass


In [109]:
sql('select * from students limit 5')

,student_id,gender,region,education,imd_band,age_band,disability
0,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,N
1,28400,F,Scotland,HE Qualification,20-30%,35-55,N
2,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,Y
3,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,N
4,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,N


In [111]:
sql('select age_band,count(*) from students group by age_band')

,age_band,count(*)
0,55<=,179
1,35-55,8500
2,0-35,20178


In [ ]:
query = '''
    update students
    set age_band = '55+'
    where age_band = "55<=";
'''
run(query)

In [119]:
sql('select * from students limit 5')

,student_id,gender,region,education,imd_band,age_band,disability
0,11391,M,East Anglian Region,HE Qualification,90-100%,55+,N
1,28400,F,Scotland,HE Qualification,20-30%,35-55,N
2,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,Y
3,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,N
4,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,N


In [120]:
sql('select imd_band,count(*) from students group by imd_band')

,imd_band,count(*)
0,90-100%,2223
1,20-30%,3209
2,30-40%,3137
3,50-60%,2794
4,80-90%,2426
5,70-80%,2536
6,None,980
7,60-70%,2567
8,40-50%,2898
9,10-20,3138


In [143]:
students = sql('select * from students')

In [122]:
select('courses')

,course_name,course_part,part_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


In [123]:
courses = sql('select * from courses')

In [125]:
sql('select * from student_assessment limit 5')

,assessment_id,student_id,submitted_after,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0


In [129]:
query = '''
    select 
    student_id,
    count(distinct assessment_id) as total_assessments,
    round(avg(submitted_after),2) as avg_submission_days,
    round(avg(score),2) as average_score
    from student_assessment
    group by student_id;
'''
student_assessment_agg = sql(query)

In [132]:
student_assessment_agg.head(5)

,student_id,total_assessments,avg_submission_days,average_score
0,6516,5,111.60,61.80
1,8462,7,23.00,87.00
2,11391,5,112.40,82.00
3,23629,4,55.75,82.50
4,23698,9,133.44,74.44


In [130]:
select('assessments')

,course_name,course_part,assessment_id,assessment_type,days,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


In [133]:
select('student_vle')

,course_name,part_name,student_id,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1


In [136]:
query = '''
    select student_id,
    count(distinct id_site) as total_vle_used,
    round(avg(date),2) as average_days,
    sum(sum_click) as total_clicks
    from student_vle
    group by student_id
'''
student_vle_agg = sql(query)

In [144]:
req_tables = [student_info,students,courses,student_assessment_agg,student_vle_agg]

In [148]:
for tables in req_tables:
    display(tables.sample(3))

,course_name,part_name,student_id,previous_attempts,studied_credits,final_result
13469,DDD,2013B,435780,1,180,Withdrawn
4447,BBB,2013J,609873,0,120,Withdrawn
14209,DDD,2013B,1622020,0,120,Withdrawn


,student_id,gender,region,education,imd_band,age_band,disability
8679,363939,M,West Midlands Region,Lower Than A Level,30-40%,0-35,N
4604,2455578,F,South Region,A Level or Equivalent,60-70%,0-35,N
28500,653495,F,Yorkshire Region,Lower Than A Level,20-30%,35-55,N


,course_name,course_part,part_length
13,EEE,2014J,269
11,DDD,2014B,241
3,BBB,2014J,262


,student_id,total_assessments,avg_submission_days,average_score
13060,601213,12,153.00,92.83
15092,621473,9,146.33,81.67
9725,569609,20,156.75,74.15


,student_id,total_vle_used,average_days,total_clicks
19366,643502,100,114.52,1774.0
16993,623326,31,0.81,167.0
11809,578953,6,21.00,9.0


In [150]:
data = (student_info.merge(students,on='student_id',how='left').
 merge(courses,left_on=['course_name','part_name'],right_on=['course_name','course_part'],how='left').
 merge(student_assessment_agg,on='student_id',how='left').
 merge(student_vle_agg,on='student_id',how='left'))

In [151]:
data.head()

,course_name,part_name,student_id,previous_attempts,studied_credits,final_result,gender,region,education,imd_band,age_band,disability,course_part,part_length,total_assessments,avg_submission_days,average_score,total_vle_used,average_days,total_clicks
0,AAA,2013J,11391,0,240,Pass,M,East Anglian Region,HE Qualification,90-100%,55+,N,2013J,268,5.0,112.4,82.0,55.0,102.13,934.0
1,AAA,2013J,28400,0,60,Pass,F,Scotland,HE Qualification,20-30%,35-55,N,2013J,268,5.0,114.2,66.4,84.0,86.99,1435.0
2,AAA,2013J,30268,0,60,Withdrawn,F,North Western Region,A Level or Equivalent,30-40%,35-55,Y,2013J,268,NaN,NaN,NaN,22.0,2.36,281.0
3,AAA,2013J,31604,0,60,Pass,F,South East Region,A Level or Equivalent,50-60%,35-55,N,2013J,268,5.0,112.2,76.0,82.0,106.15,2158.0
4,AAA,2013J,32885,0,60,Pass,F,West Midlands Region,Lower Than A Level,50-60%,0-35,N,2013J,268,5.0,125.6,54.4,66.0,91.93,1034.0


In [153]:
data.to_csv(r"C:\Users\Rano's PC\Machine\github_repo_cloned\Portfolio-Projects-Repository\Open University Dataset\Solutions\dataset.csv",index = False)